In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, TensorBoard
# from data import trainGenerator, testGenerator, saveResult, testGenerator2
import keras.backend as K
import os, cv2
import numpy as np
import skimage.io as io
import skimage.transform as trans
import tensorflow as tf

In [3]:
# Code to crop images to specific size

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
def crop_image(src, save_path):
    TEST_SET = os.listdir(src)
    img_h = 256
    img_w = 256
    stride = img_h-40
    for n in range(len(TEST_SET)):
        image_name = TEST_SET[n]
        path1 = image_name[0:-7]+'mask.png'  #rename mask
        # load the image
        # image = cv2.imread(os.path.join(src,image_name), cv2.IMREAD_UNCHANGED)
        # image = cv2.imread(os.path.join(src,image_name))
        print(os.path.join(src,image_name))
        image = io.imread(os.path.join(src,image_name))
        
        print(image.shape)
        h, w, _ = image.shape
        # h, w = image.shape

        num = 0;
        #image = img_to_array(image)
        # padding_img = (padding_img - np.min(padding_img)) / (np.max(padding_img) - np.min(padding_img))

        print('[{}/{}], croping:{}'.format(n+1, len(TEST_SET), image_name))

        #mask_whole = np.zeros((h, w, 1), dtype=np.uint8)
        #temp = np.zeros((img_h, img_h), dtype=np.uint8)

        for i in range(0, (h // stride)+1):
            for j in range(0, (w // stride)+1):
                h_begin = i * stride
                w_begin = j * stride
                
                if h_begin + img_h > h:
                    h_begin = h_begin - (h_begin + img_h - h)
                
                if w_begin + img_w > w:
                    w_begin = w_begin - (w_begin + img_w - w)
                
                crop = image[h_begin:h_begin + img_h, w_begin:w_begin + img_w] 
                if num <= 9:
                    #path1 = image_name[0:-4]+'0'+ str(num)+'.jpg'
                    path1 = image_name[0:-4]+'0'+ str(num)+'.png'
                else:
                    #path1 = image_name[0:-4]+str(num)+'.jpg'
                    path1 = image_name[0:-4]+str(num)+'.png'
                
                #io.imsave(save_path + path1, crop)
                cv2.imwrite(save_path + path1, crop)
                num = num + 1
        print('Done!')

In [5]:
import os
import shutil
# Directory containing the training data
source_directory = '../DeepGlobe_road_extraction/train'   

# Directory where PNG images will be moved
satelliteDir = "satellite_images/"
maskedDir = "masked_images/" 

# Create the subdirectory if it doesn't exist
if not os.path.exists(satelliteDir):
    os.makedirs(satelliteDir)
    
if not os.path.exists(maskedDir):
    os.makedirs(maskedDir)

# List all files in the directory
all_files = os.listdir(source_directory)

# Filter and move PNG files to the subdirectory
for file in all_files:
    if file.endswith('.png'):
        # Full path of the source file
        source_path = os.path.join(source_directory, file)
        
        # Full path of the destination file
        destination_path = os.path.join(maskedDir, file)
        
        # Move the file to the subdirectory
        shutil.copy(source_path, destination_path)
    elif file.endswith('.jpg'):
        # Full path of the source file
        source_path = os.path.join(source_directory, file)
        
        # Full path of the destination file
        destination_path = os.path.join(satelliteDir, file)
        
        # Move the file to the subdirectory
        shutil.copy(source_path, destination_path)

In [6]:
# Cropping Train Images

cropsatDir = "cropped_satellite_images/"
cropmaskDir = "cropped_masked_images/" 

if not os.path.exists(cropsatDir):
    os.makedirs(cropsatDir)
    
if not os.path.exists(cropmaskDir):
    os.makedirs(cropmaskDir)

crop_image(satelliteDir, cropsatDir)
crop_image(maskedDir, cropmaskDir)

satellite_images/100034_sat.jpg
(1024, 1024, 3)
[1/6226], croping:100034_sat.jpg
Done!
satellite_images/100081_sat.jpg
(1024, 1024, 3)
[2/6226], croping:100081_sat.jpg
Done!
satellite_images/100129_sat.jpg
(1024, 1024, 3)
[3/6226], croping:100129_sat.jpg
Done!
satellite_images/100703_sat.jpg
(1024, 1024, 3)
[4/6226], croping:100703_sat.jpg
Done!
satellite_images/100712_sat.jpg
(1024, 1024, 3)
[5/6226], croping:100712_sat.jpg
Done!
satellite_images/100773_sat.jpg
(1024, 1024, 3)
[6/6226], croping:100773_sat.jpg
Done!
satellite_images/100841_sat.jpg
(1024, 1024, 3)
[7/6226], croping:100841_sat.jpg
Done!
satellite_images/100867_sat.jpg
(1024, 1024, 3)
[8/6226], croping:100867_sat.jpg
Done!
satellite_images/100892_sat.jpg
(1024, 1024, 3)
[9/6226], croping:100892_sat.jpg
Done!
satellite_images/101225_sat.jpg
(1024, 1024, 3)
[10/6226], croping:101225_sat.jpg
Done!
satellite_images/101296_sat.jpg
(1024, 1024, 3)
[11/6226], croping:101296_sat.jpg
Done!
satellite_images/101395_sat.jpg
(1024, 10

In [7]:
def encoder_block(inputs, num_filters): 
    # Convolution with 3x3 filter followed by ReLU activation 
    x = tf.keras.layers.Conv2D(num_filters, 3,padding = 'valid')(inputs) 
    x = tf.keras.layers.Activation('relu')(x) 

    # Convolution with 3x3 filter followed by ReLU activation 
    x = tf.keras.layers.Conv2D(num_filters, 3, padding = 'valid')(x) 
    x = tf.keras.layers.Activation('relu')(x) 

    # Max Pooling with 2x2 filter 
    x = tf.keras.layers.MaxPool2D(pool_size = (2, 2), strides = 2)(x) 
    return x

In [8]:
def decoder_block(inputs, skip_features, num_filters): 
    # Upsampling with 2x2 filter 
    x = tf.keras.layers.Conv2DTranspose(num_filters, (2, 2), strides = 2, padding = 'valid')(inputs) 
      
    # Copy and crop the skip features  
    # to match the shape of the upsampled input 
    skip_features = tf.image.resize(skip_features, size = (x.shape[1], x.shape[2])) 
    x = tf.keras.layers.Concatenate()([x, skip_features]) 
      
    # Convolution with 3x3 filter followed by ReLU activation 
    x = tf.keras.layers.Conv2D(num_filters, 3, padding = 'valid')(x) 
    x = tf.keras.layers.Activation('relu')(x) 
  
    # Convolution with 3x3 filter followed by ReLU activation 
    x = tf.keras.layers.Conv2D(num_filters, 3, padding = 'valid')(x) 
    x = tf.keras.layers.Activation('relu')(x) 
      
    return x

In [9]:
def unet_model(input_shape = (256, 256, 3), num_classes = 1): 
    inputs = tf.keras.layers.Input(input_shape) 
      
    # Contracting Path 
    s1 = encoder_block(inputs, 64) 
    s2 = encoder_block(s1, 128) 
    s3 = encoder_block(s2, 256) 
    s4 = encoder_block(s3, 512) 
      
    # Bottleneck 
    b1 = tf.keras.layers.Conv2D(1024, 3, padding = 'valid')(s4) 
    b1 = tf.keras.layers.Activation('relu')(b1) 
    b1 = tf.keras.layers.Conv2D(1024, 3, padding = 'valid')(b1) 
    b1 = tf.keras.layers.Activation('relu')(b1) 
      
    # Expansive Path 
    s5 = decoder_block(b1, s4, 512) 
    s6 = decoder_block(s5, s3, 256) 
    s7 = decoder_block(s6, s2, 128) 
    s8 = decoder_block(s7, s1, 64) 
      
    # Output 
    outputs = tf.keras.layers.Conv2D(num_classes,  
                                     1,  
                                     padding = 'valid',  
                                     activation = 'sigmoid')(s8) 
      
    model = tf.keras.models.Model(inputs = inputs,  
                                  outputs = outputs,  
                                  name = 'U-Net') 
    return model 

In [12]:

def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "rgb",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (256,256),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        #img,mask = adjustData(img,mask,flag_multi_class,num_class)
        img = img / 255.0
        mask = mask / 255.0
        yield (img,mask)



In [15]:
data_gen_args = dict(rotation_range=90.,
                    #width_shift_range=0.1,
                    #height_shift_range=0.1,
                    #shear_range=0.1,
                    #zoom_range=0.1,
                    fill_mode='nearest',
                    horizontal_flip=True,
                    vertical_flip=True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor=0.2, patience=3, verbose=0, mode='min', epsilon=1e-4, 
                              cooldown=0, min_lr=1e-6)
visual = TensorBoard(log_dir='./D_resunet1_log', histogram_freq=0, write_graph=True, write_images=True)
earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=0, mode='min')

train_Gene = trainGenerator(2,'./','cropped_satellite_images/','cropped_masked_images/',data_gen_args,save_to_dir = None)
# to be fixed(train-val split)
val_Gene = trainGenerator(2,'./','sat_img_crops','masks_crops',data_gen_args)

In [18]:
model = unet_model(input_shape=(256, 256, 3), num_classes=2)

In [19]:
model_checkpoint = ModelCheckpoint('U-Net.hdf5', monitor='val_loss',verbose=1, save_best_only=True)
model.fit(train_Gene,steps_per_epoch=50,epochs=10,
                    callbacks=[model_checkpoint, visual, reduce_lr, earlystop], 
                    validation_data=val_Gene, validation_steps=220)#step_per_epoch and validation_steps equals to number of samples divide batchsize

NameError: name 'val_Gene' is not defined